### 文本表示方法 Part1

区别于结构化数据和图片数据，在自然语言领域，文本是不定长度的（每个样本的**特征数不一致**）。文本表示成计算机能够运算的数字或向量的方法一般称为**词嵌入**（Word Embedding）方法。词嵌入将不定长的文本转换到定长的空间内，是文本分类的第一步。

句子1：我爱北京天安门  
句子2：我喜欢上海

#### One-hot 方法

与数据挖掘任务中的独热码是一致的，即将每一个单词使用一个高纬度（维度数等于去重后的单词个数）的稀疏离散向量表示。具体将每个字/词（看作某个特征）编码一个索引，然后根据索引进行赋值。  

如：  
句子1：我 爱 北 京 天 安 门  
句子2：我 喜 欢 上 海

#### Bag of Words（词袋表示）

也称为Count Vectors，每个文档的字/词可以使用其出现次数来进行表示。在one-hot的基础上（相同维度的稀疏向量），标记出每个单词（特征）的出现次数。

在 **sklearn** 中可以通过 CountVectorizer 来实现这一步骤

#### N-gram

与 Bag of Words 类似，区别在于 相邻单词组合（相邻的N个） 构造新的单词，并进行计数。

如：   
句子1：我爱 爱北 北京 京天 天安 安门  
句子2：我喜 喜欢 欢上 上海

#### TF-IDF
TF-IDF 分数由两部分组成：第一部分是词语频率（Term Frequency），第二部分是逆文档频率（Inverse Document Frequency）。

TF(t)= 该词语在当前文档出现的次数 / 当前文档中词语的总数
IDF(t)= ln（语料库中文档总数 / 出现该词语的文档总数）

在 **sklearn** 中可以通过 TfidfVectorizer 来实现这一步骤

### 基于机器学习的文本分类模型
对比不同文本表示算法的精度，通过本地构建验证集计算F1得分

#### Bag of Words + RidgeClassifier

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train = pd.read_csv('./train_set.csv.zip', sep='\t', nrows=15000)

# 将文本用 BOW 方法进行表示，限制构造的向量长度！！！
vectorizer = CountVectorizer(max_features=3000)
train_BOW = vectorizer.fit_transform(train['text'])

# 前10000个数据进行训练
clf = RidgeClassifier()
clf.fit(train_BOW[:10000], train['label'].values[:10000])

# 后5000个数据进行测试
val_pred = clf.predict(train_BOW[10000:])
print(f1_score(train['label'].values[10000:], val_pred, average='macro'))

0.65441877581244


In [9]:
val_pred[:50]

array([0, 1, 0, 4, 5, 3, 1, 0, 1, 0, 3, 3, 0, 2, 1, 2, 4, 1, 1, 0, 3, 1,
       2, 1, 3, 0, 2, 0, 3, 5, 2, 1, 3, 5, 2, 6, 2, 0, 2, 2, 3, 1, 3, 0,
       8, 2, 2, 6, 2, 1], dtype=int64)

In [11]:
train.label[:50].values

array([ 2, 11,  3,  2,  3,  9,  3, 10, 12,  3,  0,  7,  4,  0,  0,  1,  1,
        2,  4,  2,  0,  1,  2,  2, 12,  4,  3,  4,  1,  1,  7,  0,  0,  3,
        1,  1,  0,  1,  1,  4,  2,  0,  3,  1,  9,  7,  2,  2,  2,  7],
      dtype=int64)

In [13]:
val_pred[:50] == train.label[:50].values

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False,  True, False,  True, False])

In [14]:
print(f1_score(train['label'].values[10000:10050], val_pred[:50], average='macro'))

0.5555191534033663
Wall time: 1.97 ms


#### TF-IDF + RidgeClassifier

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=3000)
train_TF_IDF = tfidf.fit_transform(train['text'])

# 训练
clf2 = RidgeClassifier()
clf2.fit(train_TF_IDF[:10000], train['label'].values[:10000])

# 验证
val_pred2 = clf.predict(train_TF_IDF[10000:])
print(f1_score(train['label'].values[10000:], val_pred, average='macro'))

0.65441877581244


#### 也没有数据的预处理，直接建模，但是结果为什么查这么多？？

### Baseline--使用全部数据进行训练和验证

In [15]:
train_all = pd.read_csv('./train_set.csv.zip', sep='\t')
test_all = pd.read_csv('./test_a.csv.zip', sep='\t')

#### 使用全部数据来进行文本表示
该文本表示方法是无监督学习，所以可以使用测试集数据来进行训练

In [17]:
train_text = train_all['text']
test_text = test_all['text']
all_text = pd.concat([train_text, test_text])

In [18]:
%%time
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)

word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

Wall time: 4min 44s


In [23]:
train_word_features[:2]

<2x6977 sparse matrix of type '<class 'numpy.float64'>'
	with 616 stored elements in Compressed Sparse Row format>

#### 划分训练数据为训练集和验证集

In [20]:
from sklearn.model_selection import train_test_split

X_train = train_word_features
y_train = train_all['label']
X_test = test_word_features

# 可以改变输入维度
x_train_, x_valid_, y_train_, y_valid_ = train_test_split(X_train, y_train, test_size=0.2)

#### 逻辑回归

In [22]:
%%time
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=4, n_jobs=3)
lr.fit(x_train_, y_train_)

y_pred = lr.predict(x_valid_)
train_scores = lr.score(x_train_, y_train_)
print(train_scores, f1_score(y_pred, y_valid_, average='macro'))

d:\softwares\anaconda\envs\tf\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\softwares\anaconda\envs\tf\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\softwares\anaconda\envs\tf\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 3.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


0.948625 0.9137070089196753
Wall time: 3min 8s
